In [14]:
import pandas as pd
import geopandas as gpd
import cdsapi
import xarray as xr
from urllib.request import urlopen
import datetime
import random

In [3]:
c = cdsapi.Client()

In [15]:
points_df = gpd.read_file("data/all/points_fire_no_fire_biodiversidad.geojson")
points_df

,x,y,altitud,pendiente,orientacion,is_fire,date,index_right,Poligon,ORIGEN,...,TIPOBOSQUE,n_CLAIFN,UsoIFN,n_CLAMFE,UsoMFE,n_REGBIO,RegBio,LULUCF,Shape_Area,geometry
0,635421.882389,4.377432e+06,858.38650,3.307040,46.159070,False,NaT,12626,461573,MFE50,...,-,200,Agrícola,200,Cultivos,4,Mediterránea,700,2.212515e+08,POINT (635421.882 4377432.085)
1,636866.882389,4.379702e+06,849.45123,4.176286,145.870990,False,NaT,12626,461573,MFE50,...,-,200,Agrícola,200,Cultivos,4,Mediterránea,700,2.212515e+08,POINT (636866.882 4379702.085)
2,633301.882389,4.377547e+06,838.59500,8.920355,149.516970,False,NaT,12626,461573,MFE50,...,-,200,Agrícola,200,Cultivos,4,Mediterránea,700,2.212515e+08,POINT (633301.882 4377547.085)
3,635496.882389,4.378912e+06,840.50930,2.810467,339.215300,False,NaT,12626,461573,MFE50,...,-,200,Agrícola,200,Cultivos,4,Mediterránea,700,2.212515e+08,POINT (635496.882 4378912.085)
4,637216.882389,4.380737e+06,849.42580,11.864986,35.107677,False,NaT,12626,461573,MFE50,...,-,200,Agrícola,200,Cultivos,4,Mediterránea,700,2.212515e+08,POINT (637216.882 4380737.085)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30005,725479.305975,4.268695e+06,531.25824,60.443610,176.078890,False,NaT,11819,708181,MFE50,...,-,140,Monte desarbolado,140,Desarbolado,4,Mediterránea,220,2.268380e+05,POINT (725479.306 4268695.206)
30006,725909.305975,4.268665e+06,603.67773,43.613365,314.474980,False,NaT,11819,708181,MFE50,...,-,140,Monte desarbolado,140,Desarbolado,4,Mediterránea,220,2.268380e+05,POINT (725909.306 4268665.206)
30007,722394.305975,4.261770e+06,187.02850,9.775122,326.528230,False,NaT,11087,691555,MFE50,...,-,140,Monte desarbolado,140,Desarbolado,4,Mediterránea,220,4.135418e+05,POINT (722394.306 4261770.206)
30008,723229.305975,4.262245e+06,212.96576,13.285996,283.316440,False,NaT,10357,680035,MFE50,...,-,300,Artificial,300,Artificial,4,Mediterránea,800,5.052702e+05,POINT (723229.306 4262245.206)


In [9]:
points_df.crs

<Projected CRS: EPSG:3042>
Name: ETRS89 / UTM zone 30N (N-E)
Axis Info [cartesian]:
- N[north]: Northing (metre)
- E[east]: Easting (metre)
Area of Use:
- name: Europe between 6°W and 0°W: Faroe Islands offshore; Ireland - offshore; Jan Mayen - offshore; Norway including Svalbard - offshore; Spain - onshore and offshore.
- bounds: (-6.0, 35.26, 0.0, 80.53)
Coordinate Operation:
- name: UTM zone 30N
- method: Transverse Mercator
Datum: European Terrestrial Reference System 1989
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [16]:
points_df.to_crs('EPSG:4326', inplace=True)
points_df = points_df[['altitud', 'pendiente', 'orientacion', 'is_fire', 'date','TIPOBOSQUE', 'n_CLAIFN', 'UsoIFN', 'n_CLAMFE', 'UsoMFE', 'geometry']]
points_df

,altitud,pendiente,orientacion,is_fire,date,TIPOBOSQUE,n_CLAIFN,UsoIFN,n_CLAMFE,UsoMFE,geometry
0,858.38650,3.307040,46.159070,False,NaT,-,200,Agrícola,200,Cultivos,POINT (-1.42417 39.53588)
1,849.45123,4.176286,145.870990,False,NaT,-,200,Agrícola,200,Cultivos,POINT (-1.40689 39.55610)
2,838.59500,8.920355,149.516970,False,NaT,-,200,Agrícola,200,Cultivos,POINT (-1.44881 39.53725)
3,840.50930,2.810467,339.215300,False,NaT,-,200,Agrícola,200,Cultivos,POINT (-1.42300 39.54920)
4,849.42580,11.864986,35.107677,False,NaT,-,200,Agrícola,200,Cultivos,POINT (-1.40261 39.56536)
...,...,...,...,...,...,...,...,...,...,...,...
30005,531.25824,60.443610,176.078890,False,NaT,-,140,Monte desarbolado,140,Desarbolado,POINT (-0.41300 38.53811)
30006,603.67773,43.613365,314.474980,False,NaT,-,140,Monte desarbolado,140,Desarbolado,POINT (-0.40808 38.53773)
30007,187.02850,9.775122,326.528230,False,NaT,-,140,Monte desarbolado,140,Desarbolado,POINT (-0.45056 38.47654)
30008,212.96576,13.285996,283.316440,False,NaT,-,300,Artificial,300,Artificial,POINT (-0.44085 38.48061)


In [17]:
# Generate random date times for non-fire points
start = datetime.datetime(year=1993, month=1, day=1)
end = datetime.datetime(year=2019, month=12, day=31)

r_dates =  points_df["date"] .isnull()
dates_list = [start + datetime.timedelta(seconds=random.randint(0, int((end - start).total_seconds()))) for x in r_dates if x is True]
points_df.loc[r_dates, "date"] = dates_list


In [18]:
points_df

,altitud,pendiente,orientacion,is_fire,date,TIPOBOSQUE,n_CLAIFN,UsoIFN,n_CLAMFE,UsoMFE,geometry
0,858.38650,3.307040,46.159070,False,1994-05-17 09:06:33,-,200,Agrícola,200,Cultivos,POINT (-1.42417 39.53588)
1,849.45123,4.176286,145.870990,False,2006-06-11 15:16:51,-,200,Agrícola,200,Cultivos,POINT (-1.40689 39.55610)
2,838.59500,8.920355,149.516970,False,2012-05-07 07:54:19,-,200,Agrícola,200,Cultivos,POINT (-1.44881 39.53725)
3,840.50930,2.810467,339.215300,False,2013-03-05 03:23:04,-,200,Agrícola,200,Cultivos,POINT (-1.42300 39.54920)
4,849.42580,11.864986,35.107677,False,1996-01-06 18:04:09,-,200,Agrícola,200,Cultivos,POINT (-1.40261 39.56536)
...,...,...,...,...,...,...,...,...,...,...,...
30005,531.25824,60.443610,176.078890,False,2001-10-01 10:47:54,-,140,Monte desarbolado,140,Desarbolado,POINT (-0.41300 38.53811)
30006,603.67773,43.613365,314.474980,False,2006-12-04 16:50:15,-,140,Monte desarbolado,140,Desarbolado,POINT (-0.40808 38.53773)
30007,187.02850,9.775122,326.528230,False,2004-04-10 09:59:24,-,140,Monte desarbolado,140,Desarbolado,POINT (-0.45056 38.47654)
30008,212.96576,13.285996,283.316440,False,1995-10-09 11:32:47,-,300,Artificial,300,Artificial,POINT (-0.44085 38.48061)


In [83]:
# Count the number of rows with 'is_fire' == True and 'is_fire' == False
num_fire_true = points_df['is_fire'].sum()
num_fire_false = len(points_df) - num_fire_true

# Calculate the weights for sampling
weight_fire_true = 1.0 / num_fire_true
weight_fire_false = 1.0 / num_fire_false

# Assign weights based on the value of 'is_fire'
weights = points_df['is_fire'].map({True: weight_fire_true, False: weight_fire_false})

# Sample the DataFrame using the calculated weights
sampled_df = points_df.sample(n=10000, weights=weights, replace=True, ignore_index=True)

In [84]:
sampled_df

,altitud,pendiente,orientacion,is_fire,date,TIPOBOSQUE,n_CLAIFN,UsoIFN,n_CLAMFE,UsoMFE,geometry
0,226.43176,1.512323,287.237460,False,2003-07-13 08:03:41,-,200,Agrícola,200,Cultivos,POINT (-0.35770 38.91151)
1,591.98520,37.829388,174.613740,True,2007-03-07 00:00:00,-,141,Monte desarbolado. Matorral,140,Desarbolado,POINT (-0.49885 39.91551)
2,302.96150,57.214653,76.929750,True,2007-08-28 00:00:00,Coníferas,111,Monte arbolado. Bosque,110,Arbolado,POINT (-0.15506 40.12667)
3,365.60150,36.542430,112.896065,False,2005-07-31 11:36:00,-,141,Monte desarbolado. Matorral,140,Desarbolado,POINT (-0.10809 38.78970)
4,216.34225,48.399876,129.588600,True,2016-06-15 00:00:00,-,140,Monte desarbolado,140,Desarbolado,POINT (-0.62135 39.06324)
...,...,...,...,...,...,...,...,...,...,...,...
9995,321.46176,58.597137,240.593480,False,2018-06-27 22:18:46,Coníferas,121,Monte con arbolado ralo. Bosque,120,Arbolado ralo,POINT (-0.36823 39.69124)
9996,998.84950,23.794138,231.235290,True,2012-07-12 00:00:00,Masas mixtas,171,Monte desarbolado con arbolado disperso. Arbol...,130,Arbolado disperso,POINT (-0.49897 38.73913)
9997,79.08550,39.971180,339.268550,False,2003-01-12 18:33:19,-,300,Artificial,300,Artificial,POINT (-0.16729 38.92834)
9998,288.20276,34.997368,306.008120,False,2002-02-24 19:58:32,Coníferas,111,Monte arbolado. Bosque,110,Arbolado,POINT (-0.32392 38.88786)


In [85]:
sampled_df['n_CLAIFN'].unique()

array([200, 141, 111, 140, 132, 300, 121, 171, 500, 112, 161, 400, 172,
       122, 114, 150, 142, 124])

In [86]:
len(sampled_df[sampled_df['is_fire'] == True])

4996

In [87]:
# Get meteo data
dataset = 'reanalysis-era5-pressure-levels'
download_flag = False

iteration_count = 0  # Initialize iteration count

for r in sampled_df.iterrows():
    print(r[0])
    
    params = {
         'format': 'netcdf',
         'product_type': 'reanalysis',
         'variable': ['temperature', 'u_component_of_wind', 'v_component_of_wind', 'specific_humidity', 'relative_humidity'],
         'pressure_level':'1000',
         'year':str(r[1]["date"].year),
         'month':str(r[1]["date"].month),
         'day': str(r[1]["date"].day),
         'time': [str(x) for x in range(0,24)],
         'grid': [0.25, 0.25],
         'area': [r[1]["geometry"].y, r[1]["geometry"].x, r[1]["geometry"].y, r[1]["geometry"].x],
         }
     
    # Download and load into memory    
    fl = c.retrieve(dataset, params)
    if download_flag:
         fl.download("./output.nc")
         
    with urlopen(fl.location) as f:
         ds = xr.open_dataset(f.read())         
    df = ds.to_dataframe()
    # Write values of interestintopoints datafrafe
    sampled_df.loc[r[0], ["t_max", "u", "v", 'specific_humidity', 'relative_humidity']] = df['t'].max(), df['u'].mean(), df['v'].mean(), df['q'].min(), df['r'].min()

    # Increment the iteration count
    iteration_count += 1
    
    # Check if the current iteration count is a multiple of 100
    if iteration_count % 100 == 0:
        # Save the sampled_df DataFrame to a file
        filename = f"data/all/meteo/sampled_df_{iteration_count}"
        sampled_df.to_file(f"{filename}.geojson", driver='GeoJSON')
        sampled_df.to_csv(f"{filename}.csv", index=False)

0


2023-05-23 12:12:41,379 INFO Welcome to the CDS
2023-05-23 12:12:41,380 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2023-05-23 12:12:41,490 INFO Request is queued
2023-05-23 12:12:42,600 INFO Request is running
2023-05-23 12:12:49,906 INFO Request is completed
2023-05-23 12:12:50,207 INFO Welcome to the CDS
2023-05-23 12:12:50,207 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2023-05-23 12:12:50,302 INFO Request is queued


1


2023-05-23 12:12:51,352 INFO Request is running
2023-05-23 12:12:58,626 INFO Request is completed
2023-05-23 12:12:58,993 INFO Welcome to the CDS
2023-05-23 12:12:58,993 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2023-05-23 12:12:59,110 INFO Request is queued


2


...

2023-05-23 12:16:16,202 INFO Request is running
2023-05-23 12:16:23,528 INFO Request is completed
2023-05-23 12:16:23,885 INFO Welcome to the CDS
2023-05-23 12:16:23,886 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels


25


2023-05-23 12:16:24,059 INFO Request is queued
2023-05-23 12:16:25,126 INFO Request is running
2023-05-23 12:16:32,481 INFO Request is completed
2023-05-23 12:16:33,020 INFO Welcome to the CDS
2023-05-23 12:16:33,020 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2023-05-23 12:16:33,113 INFO Request is queued


26


2023-05-23 12:16:34,171 INFO Request is running


In [70]:
sampled_df.to_file('data/all/points_data.geojson', driver='GeoJSON')

In [71]:
sampled_df.to_csv("data/all/points_data.csv", index=False)